In [1]:
# coding: utf-8
"""
DM計算用
"""
from library.filer import Filer
from library.mixture_of_unigram_model import MUM
from library.dnp import Evaluation
import glob
import numpy as np

In [2]:
list_filepath = glob.glob('files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/*.txt')
list_testfile = Filer.readdump('./files/rakuten_corpus/rakuten_corpus_master/testfile/list_sepword_label.dump')
list_testword = [row[0] for row in list_testfile]
list_label = [row[1] for row in list_testfile]

In [3]:
for path in list_filepath:
    print path
    path_rev = path.replace('files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/', '')
    Filer.writetxt([path_rev], './files/result/rakuten/experiment1/DM.txt')
    for counter in range(10):
        K = 6
        alpha = 0.01    # 初期ハイパーパラメータalpha
        beta = 0.01     # 初期ハイパーパラメータbeta
        N = 1000        # 最大イテレーション回数
        converge = 0.01 # イテレーション10回ごとに対数尤度を計算し，その差分(converge)が小さければ学習を終了する

        mum = MUM(path)
        mum.set_param(alpha, beta, K, N, converge)
        mum.learn()
    
        list_predict = []
        for row in list_testword:
            try:
                list_prob = mum.infer(row)
                list_predict.append(np.argmax(list_prob))
            except KeyError:
                list_predict.append(None)

        list_predict_rev = []
        list_label_rev = []
        for predict, label in zip(list_predict, list_label):
            if predict != None:
                list_predict_rev.append(predict)
                list_label_rev.append(label)

        eva = Evaluation()
        dict_result = eva.cal_f_measure(list_predict_rev, list_label_rev)
        Filer.writetxt([dict_result['purity'],
                        dict_result['invpurity'],
                        dict_result['fvalue']].join(' '),
                       './files/result/rakuten/experiment1/DM.txt')

files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/rakuten_preprocessed_34992.txt


NameError: global name 'list_words' is not defined